In [5]:
import os

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [6]:
assert os.environ["OPENAI_API_KEY"]

In [7]:
chunks_with_metadata = []
for text in [
    {"category": "fix", "source": "lammps_fix_nvt.txt"},
    {"category": "pair_style", "source": "lammps_pair_style.txt"},
]:
    loader = TextLoader(text["source"])
    documents = loader.load()
    text_splitter = CharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100, separator="\n"
    )
    chunks = text_splitter.split_documents(documents)
    chunks_with_metadata += chunks

In [8]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=chunks_with_metadata, embedding=embeddings, persist_directory="./data"
)

/tmp/ipykernel_14192/2979001685.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model_name="gpt-4o"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

/tmp/ipykernel_14192/4061062410.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0, model_name="gpt-4o"),


In [11]:
query = "LAMMPSで圧浴としてMTKを用いたNPTアンサンブルでシミュレーションを行うコードを書いてください。"
result = qa_chain({"query": query})

/tmp/ipykernel_14192/3641214514.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


In [12]:
result

{'query': 'LAMMPSで圧浴としてMTKを用いたNPTアンサンブルでシミュレーションを行うコードを書いてください。',
 'result': '以下は、LAMMPSでMTK（Martyna-Tobias-Klein）を用いたNPTアンサンブルでシミュレーションを行うための基本的なスクリプトの例です。このスクリプトは、システムの初期化、力場の設定、NPTアンサンブルでのシミュレーションを行うためのコマンドを含んでいます。\n\n```lammps\n# LAMMPS input script for NPT ensemble using MTK barostat\n\nunits           real\natom_style      atomic\n\n# Read initial configuration\nread_data       data.lammps\n\n# Define interatomic potential\npair_style      lj/cut 10.0\npair_coeff      * * 0.1 3.5\n\n# Define settings for NPT ensemble\nfix             1 all npt temp 300.0 300.0 100.0 iso 1.0 1.0 1000.0\n\n# Output settings\nthermo          100\ndump            1 all atom 100 dump.lammpstrj\n\n# Run the simulation\ntimestep        1.0\nrun             10000\n```\n\nこのスクリプトでは、以下の設定を行っています：\n\n- `units real`：実際の物理単位を使用します。\n- `atom_style atomic`：原子スタイルを指定します。\n- `read_data data.lammps`：初期構造を読み込みます。\n- `pair_style lj/cut 10.0` と `pair_coeff * * 0.1 3.5`：Lennard-Jonesポテンシャルを使用し、カットオフ距離を10.0に設定します。\n- `

In [23]:
from langchain.evaluation import load_evaluator


def evaluate_rag_performance(question, ground_truth):
    evaluator = load_evaluator("qa")

    result = qa_chain({"query": question})
    generated_answer = result["result"]
    retrieved_docs = result["source_documents"]

    # 評価
    examples = [{"query": question, "answer": ground_truth}]
    predictions = [{"query": question, "result": generated_answer}]
    evaluation_result = evaluator.evaluate(
        examples=examples,
        predictions=predictions,
        question_key="query",
        answer_key="answer",
        prediction_key="result",
    )

    return {
        "question": question,
        "ground_truth": ground_truth,
        "generated_answer": generated_answer,
        "retrieved_docs": [doc.page_content for doc in retrieved_docs],
        "evaluation_result": evaluation_result,
    }

In [26]:
questions_and_answers = [
    {
        "question": "圧浴のMTKとは何ですか？",
        "ground_truth": "Martyna, Tobias and Kleinらによって開発された圧浴です。",
    },
]

In [27]:
results = []
for qa in questions_and_answers:
    result = evaluate_rag_performance(qa["question"], qa["ground_truth"])
    print(result)
    results.append(result)

for res in results:
    print("\n====================")
    print(f"Question: {res['question']}")
    print(f"Ground Truth: {res['ground_truth']}")
    print(f"Generated Answer: {res['generated_answer']}")
    print(f"Evaluation Result: {res['evaluation_result']}")
    print(f"Retrieved Docs: {res['retrieved_docs']}")

WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.


{'question': '圧浴のMTKとは何ですか？', 'ground_truth': 'Martyna, Tobias and Kleinらによって開発された圧浴です。', 'generated_answer': '圧浴のMTKとは、Martyna, Tobias, Kleinによって提案された圧力制御の方法を指します。これは、分子動力学シミュレーションにおいて、系の圧力を制御するための手法で、特に等温等圧（NPT）アンサンブルで使用されます。MTK法は、システムの体積を動的に調整することで、目標とする圧力を維持することを目的としています。この方法は、システムの温度と圧力を同時に制御するために、サーモスタットとバロスタットを組み合わせて使用します。', 'retrieved_docs': ['The equations of motion used are those of Shinoda et al in (Shinoda), which combine the hydrostatic equations of Martyna, Tobias and Klein in (Martyna) with the strain energy proposed by Parrinello and Rahman in (Parrinello). The time integration schemes closely follow the time-reversible measure-preserving Verlet and rRESPA integrators derived by Tuckerman et al in (Tuckerman).\nThe thermostat parameters for fix styles nvt and npt are specified using the temp keyword. Other thermostat-related keywords are tchain, tloop and drag, which are discussed below.', 'The thermostatting and barostatting is achieved by adding some dynamic variables